In [ ]:
import my.workouts.sources.endomondo as end
data = end.get_endomondo()
import pandas as pd

In [ ]:
data = [d for d in data if not isinstance(d, end.ErrorT)]

In [ ]:
oth = [w for w in data if w.raw.sport == 'Other']
oth = [w for w in oth if w.minutes < 15]
oth = [w for w in oth if len(w.points) > 10]
# TODO wonder if it's possible to import ipython notebooks for various analysis? probably not..

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def func(x, a, b, c): 
     return a * np.exp(-b * x) + c

def func(x, a, b): 
     return a * np.exp(-b * x) + 60
    
sz = (12, 10)
plt.figure(figsize=sz)    
    
def plot_one(series):
    print(f"plotting {len(series)}")
#     secs   = np.array([d.seconds               for d in deltas])

    # TODO would be interesting to track after what workout is that happening
    # TODO smoothen first??
    series = series.rolling('20s').mean()
    # series.plot(figsize=sz)

    idx = series.index
    secs = np.array([x.seconds for x in idx])
    hrs  = np.array([h         for h in series])
    
    #  polyfit is not great... e.g. it can't go below certain level..
    # pp = np.poly1d(np.polyfit(secs, hrs, deg=2))
    # print(pp)
    popt, pcov = curve_fit(func, secs, hrs, maxfev=10000)
    # print(popt)

    ff = lambda x: func(x, *popt)
    
    # TODO err what's up with simultaneous pandas plot??

    [pl] = plt.plot(secs, hrs)
    color = pl.get_color()
    plt.plot(secs, ff(secs), color=color)
    # x_new = np.linspace(x[0], x[-1], 50)
    # y_new = f(x_new)
    # TODO shit. yeah, unlikely it's time independent?

    
# plt.plot(data0.x, data0.y)
# for target in [data1, data2]:
#     dx = np.mean(np.diff(data0.x.values))
#     shift = (np.argmax(signal.correlate(data0.y, target.y)) - len(target.y)) * dx
#     plt.plot(target.x + shift, target.y)    
    # series.plot()
from scipy import signal

def run():
    dss = []
    for o in oth:
        points = [p for p in o.points if p.hr is not None]
        hrs    = np.array([p.hr           for p in points])
        deltas = [p.time - points[0].time for p in points]
        ss = pd.Series(hrs, index=deltas)
        ss = ss.resample('1S').mean().interpolate()
        dss.append(ss)
        
    ref = dss[-3].copy()
    
    def find_shift(aa, bb):
        # assumes that bb < aa?
        norms = []
        for shift in range(len(aa) - len(bb)):
            overlap = aa[shift: shift + len(bb)] - bb
            norms.append(np.linalg.norm(overlap))
        return np.argmin(np.array(norms))
    
    # plot_one(ref)
    for ss in dss:
        corr = signal.correlate(ss, ref)
        # print(signal.correlate(ss, ref) / 10000)
        # data0x = np.array(range(0 , 20))
        # targex = np.array(range(0 , 10))
        data0y = np.array(range(0 , 20))
        targey = np.array(range(5 , 15))
        # plt.plot(data0x, data0y)
        
        # print(data0y - np.mean(data0y))
        # print(targey - np.mean(targey))

        # data0y = data0y - np.mean(data0y)
        # targey = targey - np.mean(targey)
        
        #plt.plot(np.correlate(data0y - np.mean(data0y), targey - np.mean(targey), mode='full'))
        # shift = find_shift(data0y, targey)
        # print(len(ref))
        # print(len(ss))
        if len(ss) >= len(ref):
            continue
        shift = find_shift(np.array(ref), np.array(ss))
        ss.index = shift + ss.index
        # corr = np.convolve(targey, data0y, mode='valid')
        # print(corr)
        # print(len(corr))
        # print(np.argmax(corr))
        # continue
        
        # plt.plot(targex, targey)
        # plt.plot(data0x, data0y)
        # continue
        # ss.index = ss.index + maxi
        # ss = ss - ss.mean()
        # plot_one(ref)
        plot_one(ss)
    # shift = maxi - len(ref2)
    # print(shift)
    # for ss in dss[-5:]:
    #     plot_one(ss)
        
run()